# VAT JDP - Few shot learning via masked LM
by Alex Periti, Michele Faedi, Francesco Mistri

A study on few shot learning through a masked LMs. The approach is applied to an italian judicial VAT dataset consisting of a few hundred samples (Galli et al., 2022). An italian cased masked LLM (namely, Umberto) is trained and evaluated on different prompt templates in order to observe the importance of prompt design. Prompts are handwritten. The work is inspired by similar related works (although more sophisticated) like (Gao et al., 2020).

The notebook was designed to work on Google Colab.

In [ ]:
!pip install "vjp[fewshot] @ git+https://github.com/Ball-Man/vjp-ita"

  Cloning https://github.com/Ball-Man/vjp-ita to /tmp/pip-install-jd13072s/vjp_d9ea98913dde4c77879ccfd6f8038218
  Running command git clone --filter=blob:none --quiet https://github.com/Ball-Man/vjp-ita /tmp/pip-install-jd13072s/vjp_d9ea98913dde4c77879ccfd6f8038218
  Resolved https://github.com/Ball-Man/vjp-ita to commit 5483d27ce311d378a6af12d558aa6701b43bfad9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# %env WANDB_PROJECT=vjp-ita
# %env WANDB_API_KEY=d49faa47a9e0233a3e3eb1970c92fe6c1dd47b13

In [ ]:
from typing import Tuple

import torch
import numpy as np
import pandas as pd
from transformers import (AutoTokenizer, AutoModelForMaskedLM, pipeline,
                          Trainer, TrainingArguments, EarlyStoppingCallback)
from torch.utils.data import Dataset
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.dummy import DummyClassifier
from functools import partial
from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch
import wandb

from vjp import data, text

Similarly to (Logan IV et al., 2022), we define (semi-quoted):
* A pre-trained masked LM (Umberto), with $T$ denoting its
vocabulary.
* A small set of training inputs $x_i \in X$ and their
corresponding labels $y_i \in Y$.
* A pattern $P : X \to T^∗$ that maps inputs to cloze
questions containing a single `<mask>` token. We call this pattern "template". The resulting sequence: $P(X)$ "prompt".
* A verbalizer $v : Y \to T$ that maps
each label to a single vocabulary token.

Our simple hand-designed verbalizer is:
* $v(0) = \text{respinto}$
* $v(1) = \text{accolto}$

The first prompt taken into consideration is the null prompt: $P(x_i) = x_i\;\text{<mask>}$.

In [ ]:
verbalizer = ('respinto', 'accolto')
# Pair of strings, one is completed with the label and one with the document
template = ("<document>", "<mask>")


def get_prompt(document_string, label, template=template,
               verbalizer=verbalizer) -> Tuple[str, str, str]:
    """Build the main components for a prompt.

    Return triple in the form: ``(first_segment, masked_segment,
    target_segment)``."""
    document_template, label_template = template
    document_text = document_template.replace('<document>', document_string)

    label_text = verbalizer[label]

    return (document_template.replace('<document>', document_string),
            label_template,
            label_template.replace('<mask>', label_text))

Data is loaded into a dataframe. The dataframe exposes two features: the preliminaries (requests, claims, arguments, ...) and the decisions (court motivations, decisions, ...). This separation is used to furtherly inspect the effect of the presence/asbsence of such information in the given input.

Labels are intuitively: 0 for rejections, 1 for upholds.

In [ ]:
upheld_docs, rejected_docs = data.load_second_instance_labeled()
df = data.shot_based_dataframe(upheld_docs, rejected_docs)
df[['preliminaries', 'decisions']] = \
    df[['preliminaries', 'decisions']].applymap(
        text.shot_normalize_whites_pipeline)
df.head()

,preliminaries,decisions,label
0,REPUBBLICA ITALIANA\nIN NOME DEL POPOLO ITALIA...,L'appello è fondato e va accolto.Preliminarmen...,1
1,REPUBBLICA ITALIANA\nIN NOME DEL POPOLO ITALIA...,Si premette che già nel PVC la Guardia di Fina...,1
2,REPUBBLICA ITALIANAIN NOME DEL POPOLO ITALIANO...,"Osserva questa Commissione, con riguardo al pr...",1
3,REPUBBLICA ITALIANA IN NOME DEL POPOLO ITALIAN...,Osserva questo collegio che dalla lettura degl...,1
4,REPUBBLICA ITALIANA\nIN NOME DEL POPOLO ITALIA...,Questa Commissione rileva che non è controvers...,1


## Baselines
Some simple baselines are computed: majority class and random.

In [ ]:
class DummyDataset(Dataset):
    """Dummy dataset, used for baselines.

    It provides the given labels as both features and targets.
    Correct features are not needed by the dummy baselines.
    This simple wrapper is used in order to obtain a split identical
    to the one carried out later on the real data.
    """

    def __init__(self, labels):
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.labels, self.labels

dummy_dataset = DummyDataset(df.label.to_numpy())

_, dummy_validation_set, dummy_test_set = torch.utils.data.random_split(
    dummy_dataset, [0.4, 0.3, 0.3], torch.Generator().manual_seed(17))
dummy_X, dummy_y = dummy_validation_set[:]

majority_baseline = DummyClassifier()
random_baseline = DummyClassifier(strategy='uniform', random_state=42)

In [ ]:
# Fit and predict on the same data, it doesn't really matter here
majority_baseline.fit(dummy_X,
                      dummy_y)
majority_preds = majority_baseline.predict(dummy_X)
print(classification_report(dummy_y, majority_preds))

              precision    recall  f1-score   support

           0       0.58      1.00      0.74       128
           1       0.00      0.00      0.00        91

    accuracy                           0.58       219
   macro avg       0.29      0.50      0.37       219
weighted avg       0.34      0.58      0.43       219



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
random_baseline.fit(dummy_X,
                      dummy_y)
random_preds = random_baseline.predict(dummy_X)
print(classification_report(dummy_y, random_preds))

              precision    recall  f1-score   support

           0       0.62      0.54      0.58       128
           1       0.45      0.53      0.48        91

    accuracy                           0.53       219
   macro avg       0.53      0.53      0.53       219
weighted avg       0.55      0.53      0.54       219



## Tokenization
Tokenization happens through Umberto's tokenizer, provided by Hugging Face's transformers. Since the max input size of the model can be very prohibitive we have paid special attention towards truncation. Truncating happens from the left, as we estimate that the most meaningful parts of the documents are usually at their end.

In [ ]:
def template_tokenized_length(tokenizer, template) -> int:
    """Return total token length of the given template."""
    template_tokenized_pre = tokenizer(template[0].replace('<document>', ''),
                                add_special_tokens=False)
    template_tokenized_post = tokenizer(template[1],
                                add_special_tokens=False)
    template_tokenized_pre_length = len(template_tokenized_pre.input_ids)
    template_tokenized_post_length = len(template_tokenized_post.input_ids)
    return (template_tokenized_pre_length + template_tokenized_post_length)


def prepare_tokenization_dataframe(tokenizer,
                                   documents: pd.DataFrame,
                                   template,
                                   columns=['preliminaries'],
                                   max_length=510) -> pd.DataFrame:
    """Return a dataframe ready to be tokenized and fed to the model.

    Columns:

    - document: a truncated sample, with a partially applied template.
    - masked: second part of the template with the mask token.
    - target: second part of the template, but the mask token is replaced
        by the correct answer from the verbalizer (based on the label).

    """
    template_length = template_tokenized_length(tokenizer, template)

    inputs = documents[columns].sum(axis=1).to_list()

    tokenized_documents_truncated_ids = tokenizer(
        inputs,
        add_special_tokens=False,
        max_length=max_length - template_length - 4,
        truncation=True,
        return_attention_mask=False).input_ids

    tokenized_documents_truncated = tokenizer.batch_decode(
        tokenized_documents_truncated_ids)

    return pd.DataFrame(
        [get_prompt(prelim, label, template)
         for prelim, label in zip(tokenized_documents_truncated, df.label)],
        columns=['document', 'masked', 'target'])


def tokenize_prompts(tokenizer, prompts_df):
    """Given a dataframe, tokenize all samples.

    The dataframe shall be produced by :func:`prepare_tokenization_dataframe`.
    """
    prompts_df_documents = prompts_df.document.to_list()
    prompts_df_masked = prompts_df.masked.to_list()
    prompts_df_target = prompts_df.target.to_list()

    tokenized_prompts = tokenizer(
        prompts_df_documents,
        text_pair=prompts_df_masked,
        text_target=prompts_df_documents,
        text_pair_target=prompts_df_target,
        add_special_tokens=True,
        padding='longest',
        return_tensors='pt')

    # Mask loss for all tokens except <mask>
    tokenized_prompts.labels[tokenized_prompts.input_ids
                             != tokenizer.mask_token_id] = -100
    return tokenized_prompts

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'Musixmatch/umberto-commoncrawl-cased-v1')
tokenizer.truncation_side = 'left'

In [ ]:
prompts_df = prepare_tokenization_dataframe(tokenizer, df, template)
prompts_df.head()

,document,masked,target
0,REPUBBLICA ITALIANA IN NOME DEL POPOLO ITALIAN...,<mask>,accolto
1,"struttura organizzativa, senza la quale non av...",<mask>,accolto
2,"di anticipo del prezzo che essa conteneva, all...",<mask>,accolto
3,"immobile, come quelle allegate nel processo pe...",<mask>,accolto
4,"di decadenza, di proporre istanza per il rimbo...",<mask>,accolto


In [ ]:
tokenized_prompts = tokenize_prompts(tokenizer, prompts_df)

## Hyperparameter search
During initial tests, we noticed an extremely high variance in the results. Manual tuning was very hard to manage, hence we decided to employ an hyperparameter search. In particular, bayesian optimization.

A simple torch `Dataset` wrapper is built and used to split the data. Data is splitted in train (88 samples), validation (66 samples) and test (65 samples) with seed 17.

In [ ]:
class VJPDataset(Dataset):
  def __init__(self, tokenized_data):
    self.tokenized_data = tokenized_data

  def __len__(self):
    return len(self.tokenized_data.attention_mask)

  def __getitem__(self, index):
    return {'input_ids': self.tokenized_data.input_ids[index],
            'attention_mask': self.tokenized_data.attention_mask[index],
            'labels': self.tokenized_data.labels[index]}

dataset = VJPDataset(tokenized_prompts)

In [ ]:
train_set, validation_set, test_set = torch.utils.data.random_split(
    dataset, [0.4, 0.3, 0.3], torch.Generator().manual_seed(17))

print(len(train_set), len(validation_set), len(test_set))

88 66 65


The verbalizer needs tokenization as well in order to nimbly compute the metrics.

In [ ]:
verbalizer_tokenized = tokenizer(verbalizer, add_special_tokens=False).input_ids
verbalizer_tokenized = tuple(map(lambda t: t[-1], verbalizer_tokenized))
print(verbalizer_tokenized, verbalizer)

(24939, 11156) ('respinto', 'accolto')


Umberto is a relatively small LLM (110M parameters), but still too big to be fine tuned completely on a Google Colab with free plan. For this reason, the majority of the layers are frozen.

In [ ]:
def model_init():
    """Download pretrained model and freeze some layers.

    Used by Hugging Face's Trainer class.
    """
    umberto = AutoModelForMaskedLM.from_pretrained(
        'Musixmatch/umberto-commoncrawl-cased-v1')

    # Freeze some layers to prevent OOM
    freeze_threshold = 150
    num = 0
    for param in umberto.base_model.parameters():
        param.requires_grad = False

        num += 1
        if num > freeze_threshold:
            break

    return umberto

During training three metrics are computed: accuracy, F1 score and the so called wrong tokens ratio. This last metric is used to monitor the amount of predicted tokens which do not fit into the verbalizer definition i.e. tokens that are not in $\mathcal{I}(v)$. This could happen since the model retains its final classification layer, with a vocabulary size of ~30k. Clearly, given the fine tuning on a very "narrow" verbalizer, it is expected to rarely predict other tokens.

In [ ]:
def metrics(eval):
    """Compute a dictionary of metrics.

    Accuracy, F1 score, wrong tokens ratio.

    Used by Hugging Face's Trainer class.
    """
    # eval.predictions, eval.label_ids
    labels_positions = eval.label_ids != -100
    target = eval.label_ids[labels_positions]
    class_target = target == verbalizer_tokenized[1]

    pred = np.argmax(eval.predictions, -1)[labels_positions]
    class_pred = pred == verbalizer_tokenized[1]

    wrong_tokens = ~np.isin(pred, verbalizer_tokenized)
    class_pred[wrong_tokens] = ~class_target[wrong_tokens]

    return {'accuracy': accuracy_score(class_target, class_pred),
            'f1_macro': f1_score(class_target, class_pred, average='macro'),
            'wrong_tokens_ratio': wrong_tokens.sum() / wrong_tokens.shape[0]}

In [ ]:
train_args = TrainingArguments('parameter_search',
                               evaluation_strategy='epoch',
                               save_strategy='epoch',
                               # load_best_model_at_end=True,
                               logging_steps=1,
                               save_total_limit=1,
                               report_to="wandb")

trainer = Trainer(args=train_args, train_dataset=train_set,
                  eval_dataset=validation_set,
                  compute_metrics=metrics,
                  model_init=model_init)

The parameter search is a bayesian optimization with F1 score as objective. The parameter space is defined as:
* Learning rate $\in [10^{-6}, 10^{-4}]$
* Train epochs $\in [3, 16]$
* Batch size $\in [4, 32]$
* Seed $\in [1, 40]$

The seed is used mostly to provide diversity in data seeds during training.

The search explores a total of 50 points in the hyperparameter space. Bayesian optimization itself is mostly deterministic, but the first points are randomly sampled. For this reason the random seed of the search is set to 42.

In [ ]:
def hp_space(trial):
  return {
        "learning_rate": tune.uniform(1e-6, 1e-4),
        "num_train_epochs": tune.uniform(3, 16),
        "seed": tune.uniform(1, 40),
        "per_device_train_batch_size": tune.uniform(4, 32),
    }

In [ ]:
# Commented to prevent the search from running multiple times
# trainer.hyperparameter_search(
#     search_alg = BayesOptSearch(metric="objective", mode="max"),
#     hp_space = hp_space,
#     direction = 'maximize',
#     backend = 'ray',
#     n_trials = 50,
#     compute_objective = lambda m: m['eval_f1_macro']
# )

Finally, the best hyperparameters are condensed here and will be used by all subsequent experiments.

In [ ]:
best_train_args = TrainingArguments('train', per_device_train_batch_size=4,
                                    evaluation_strategy='epoch',
                                    save_strategy='epoch',
                                    load_best_model_at_end=True,
                                    metric_for_best_model='eval_f1_macro',
                                    num_train_epochs=16,
                                    logging_steps=1,
                                    save_total_limit=1,
                                    learning_rate=0.00006681487360447851,
                                    report_to="wandb")
early_stop = EarlyStoppingCallback(3, 0.001)

## On different templates
Four different templates are evaluated, given the hyperparameters found from the search. Due to the high instabilities during training, each experiment is executed on five different seeds. Metrics are averaged across the five runs.

By inspecting the cells below it is possible to see the actual structure of the templates. A summary of their characteristics:
* Long template: a verbose template with an initial explanatory paragraph as well as a trailing sentence to introduce the mask.
* Short template: similar to the long one, but without the heading paragraph.
* Extra short tempate: similar to the short one, but with a shorter version of the trailing sentence.
* Null template: $P(x_i) = x_i\;\text{<mask>}$

In [ ]:
seeds = [1, 13, 56, 78, 100]

In [ ]:
my_metrics = []
for seed in seeds:
    wandb.init(group='template_short', name=f'template_short_{seed}')

    template = (
        "<document>",
        "Leggendo il documento, l'appello è <mask>"
    )

    best_train_args.seed = seed

    prompts_df = prepare_tokenization_dataframe(tokenizer, df, template)
    tokenized_prompts = tokenize_prompts(tokenizer, prompts_df)
    dataset = VJPDataset(tokenized_prompts)
    train_set, validation_set, test_set = torch.utils.data.random_split(
        dataset, [0.4, 0.3, 0.3], torch.Generator().manual_seed(17))

    trainer = Trainer(args=best_train_args, train_dataset=train_set,
                      eval_dataset=validation_set,
                      compute_metrics=metrics,
                      model_init=model_init)
    trainer.train()
    my_metrics.append(trainer.predict(validation_set).metrics)
    gc.collect()
    torch.cuda.empty_cache()

metrics_df = pd.DataFrame(my_metrics)
print(metrics_df)
print(metrics_df.describe())

wandb: Currently logged in as: ballman. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,0.725500,0.714486,0.621212,0.383178,0.000000
2,1.917100,1.378628,0.378788,0.274725,0.000000
3,0.611400,0.816981,0.590909,0.403415,0.000000
4,1.182100,1.121141,0.621212,0.383178,0.000000
5,0.372600,0.887656,0.409091,0.405680,0.000000
6,0.078300,1.568438,0.454545,0.454044,0.000000
7,1.383800,1.516659,0.469697,0.463664,0.000000
8,0.042100,1.434690,0.454545,0.399393,0.000000
9,0.004100,1.808851,0.454545,0.410714,0.000000
10,0.004400,2.401767,0.530303,0.477395,0.000000


eval/accuracy,█▁▇█▂▃▄▃▃▅▄▅▅▆▄▄
eval/f1_macro,▅▁▅▅▅▇▇▅▅█▇▇▇█▇▇
eval/loss,▁▃▁▂▂▄▄▃▅▇▆▇▇███
eval/runtime,▅▄▆▄▄▂▂▂▁▁▁▄█▃▅▅
eval/samples_per_second,▃▄▃▅▄▆▆▇██▇▅▁▅▃▄
eval/steps_per_second,▃▄▃▅▄▆▆▇▇█▇▅▁▅▃▄
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▆█▅▅▄▇▄▃▅▄▅▄▂▂▃▂▃▂▂▂▂▄▁▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,1.031500,0.835433,0.378788,0.274725,0.000000
2,0.849800,0.871595,0.621212,0.383178,0.000000
3,0.740600,0.770829,0.575758,0.422500,0.000000
4,0.595700,0.820376,0.606061,0.463750,0.000000
5,1.075500,1.216365,0.590909,0.403415,0.000000
6,0.301800,1.216619,0.575758,0.497826,0.000000
7,0.210800,1.405684,0.424242,0.415657,0.000000
8,0.041800,1.863965,0.590909,0.544828,0.000000
9,0.045600,2.350919,0.515152,0.498575,0.000000
10,0.012200,2.895728,0.515152,0.484878,0.000000


eval/accuracy,▁█▇█▇▇▂▇▅▅▆▆█▇▇▇
eval/f1_macro,▁▄▅▆▄▇▄█▇▆▆▇████
eval/loss,▁▁▁▁▂▂▃▄▅▆▇▇████
eval/runtime,█▇█▇▆▅▁▁▁▁▃▆▆▆▇▆
eval/samples_per_second,▁▂▁▂▃▄████▅▃▃▃▂▃
eval/steps_per_second,▁▂▁▂▃▄████▅▃▃▃▂▃
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▃▄█▂▆▃▄▅▄▅▆▃▄▇▁▃▂█▃▁▁▁▁▄▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,0.543700,0.707215,0.530303,0.417923,0.000000
2,0.399200,0.942495,0.424242,0.388293,0.000000
3,0.196300,0.827546,0.621212,0.383178,0.000000
4,0.999600,1.019020,0.424242,0.366026,0.000000
5,0.580300,1.294809,0.606061,0.377358,0.000000
6,0.157300,1.390237,0.606061,0.439216,0.000000
7,0.353300,1.450557,0.560606,0.487001,0.000000
8,0.009000,2.090814,0.530303,0.521404,0.000000
9,0.012200,2.740307,0.530303,0.521404,0.000000
10,0.000100,2.986210,0.575758,0.549268,0.000000


   test_loss  test_accuracy  test_f1_macro  test_wrong_tokens_ratio  \
0   2.742323       0.545455       0.488636                      0.0   
1   2.057148       0.606061       0.533696                      0.0   
2   1.662536       0.560606       0.552281                      0.0   
3   3.501660       0.621212       0.557759                      0.0   
4   3.791934       0.590909       0.574397                      0.0   

   test_runtime  test_samples_per_second  test_steps_per_second  
0        6.8035                    9.701                  1.323  
1        7.3602                    8.967                  1.223  
2        7.8198                    8.440                  1.151  
3        7.1427                    9.240                  1.260  
4        6.9556                    9.489                  1.294  
       test_loss  test_accuracy  test_f1_macro  test_wrong_tokens_ratio  \
count   5.000000       5.000000       5.000000                      5.0   
mean    2.751120       0.58

In [ ]:
my_metrics = []
for seed in seeds:
    wandb.init(group='template_long', name=f'template_long_{seed}')

    template = (
        "Il documento dell'appello è il seguente testo:\n<document>",
        "Leggendo il documento, l'appello è <mask>"
    )

    best_train_args.seed = seed
    prompts_df = prepare_tokenization_dataframe(tokenizer, df, template)
    tokenized_prompts = tokenize_prompts(tokenizer, prompts_df)
    dataset = VJPDataset(tokenized_prompts)
    train_set, validation_set, test_set = torch.utils.data.random_split(
        dataset, [0.4, 0.3, 0.3], torch.Generator().manual_seed(17))

    trainer = Trainer(args=best_train_args, train_dataset=train_set,
                      eval_dataset=validation_set,
                      compute_metrics=metrics,
                      model_init=model_init)

    trainer.train()
    my_metrics.append(trainer.predict(validation_set).metrics)
    gc.collect()
    torch.cuda.empty_cache()

metrics_df = pd.DataFrame(my_metrics)
print(metrics_df)
print(metrics_df.describe())

eval/accuracy,▅▁█▁▇▇▆▅▅▆▃▄▇▇▅▆
eval/f1_macro,▃▂▂▁▁▃▅▆▆▇▅▆██▇▇
eval/loss,▁▁▁▂▂▂▃▄▅▆██▇███
eval/runtime,█▆▆▂▁▂▁▁▄▆▆▆▅▄▄▃
eval/samples_per_second,▁▃▂▇█▇█▇▅▃▃▂▃▅▄▆
eval/steps_per_second,▁▃▂▇█▇█▇▅▃▃▂▃▅▄▆
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,██▄▄▂▄▄▃▄▆▂▃▃▂▂▃▁▃▂▁▂▁▂▁▅▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,0.630200,0.841464,0.393939,0.318182,0.000000
2,1.919500,1.023131,0.378788,0.274725,0.000000
3,0.431300,0.856111,0.621212,0.383178,0.000000
4,1.452900,1.335114,0.621212,0.383178,0.000000
5,0.312200,1.015702,0.393939,0.384902,0.000000
6,0.214300,1.185968,0.424242,0.415657,0.000000
7,0.200300,3.387443,0.393939,0.332659,0.000000
8,0.090500,2.356630,0.515152,0.492308,0.000000
9,0.222000,3.869474,0.636364,0.505000,0.000000
10,0.012800,3.499803,0.424242,0.419444,0.000000


eval/accuracy,▁▁██▁▂▁▅█▂▅▄▄▃▅▅
eval/f1_macro,▂▁▄▄▄▅▃██▅█▇▇▆▇█
eval/loss,▁▁▁▂▁▂▆▄▇▆▆▇████
eval/runtime,█▆▄▁▁▁▂▄▅▇▇▆▆▅▅▃
eval/samples_per_second,▁▃▅███▇▅▄▂▂▃▃▃▄▆
eval/steps_per_second,▁▃▅███▇▅▄▂▂▃▃▃▄▆
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▃▂▃▁▃▄▂▂▂▃▃▂▂▁▁▁█▁▁▂▃▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,0.955800,0.746277,0.393939,0.379699,0.000000
2,0.783000,0.735226,0.606061,0.377358,0.000000
3,0.570000,0.846283,0.439394,0.408860,0.000000
4,0.884500,0.899231,0.469697,0.454545,0.000000
5,0.381300,1.121723,0.500000,0.472767,0.000000
6,0.364800,1.539503,0.575758,0.422500,0.000000
7,0.127200,2.106258,0.424242,0.423713,0.000000
8,0.000400,2.487497,0.590909,0.508685,0.000000
9,0.319800,2.927325,0.590909,0.475110,0.000000
10,0.539000,2.701135,0.560606,0.511111,0.000000


eval/accuracy,▁█▂▃▄▇▂▇▇▆▅▅▅▅▅▅
eval/f1_macro,▁▁▃▅▆▃▃█▆████▇▇█
eval/loss,▁▁▁▁▂▃▄▅▆▅▇▇▇███
eval/runtime,▃▃▄▆▇▇█▆▅▄▂▁▂▁▁▂
eval/samples_per_second,▅▆▅▃▂▂▁▃▃▅▇█▇▇█▇
eval/steps_per_second,▅▆▅▃▂▂▁▃▃▅▇█▇▇█▇
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▃▅▅▃▄▂▇▃▄▂▂▂▄▄▂▁▁█▃▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,0.482600,0.777549,0.378788,0.361039,0.000000
2,0.560800,0.744624,0.621212,0.447606,0.000000
3,0.204700,0.786904,0.590909,0.403415,0.000000
4,0.939100,0.907866,0.484848,0.472744,0.000000
5,2.524700,1.555961,0.621212,0.383178,0.000000
6,0.230200,1.076410,0.590909,0.371429,0.000000
7,0.498300,0.986167,0.590909,0.454545,0.000000
8,0.102400,1.126764,0.500000,0.494312,0.000000
9,0.513400,2.063812,0.500000,0.499885,0.000000
10,0.001700,1.973256,0.500000,0.443678,0.000000


   test_loss  test_accuracy  test_f1_macro  test_wrong_tokens_ratio  \
0   3.869474       0.636364       0.505000                      0.0   
1   2.350006       0.530303       0.496926                      0.0   
2   3.040959       0.484848       0.482949                      0.0   
3   3.394127       0.530303       0.516883                      0.0   
4   3.184608       0.575758       0.511111                      0.0   

   test_runtime  test_samples_per_second  test_steps_per_second  
0        6.9217                    9.535                  1.300  
1        7.7931                    8.469                  1.155  
2        7.0921                    9.306                  1.269  
3        6.9689                    9.471                  1.291  
4        6.9742                    9.463                  1.290  
       test_loss  test_accuracy  test_f1_macro  test_wrong_tokens_ratio  \
count   5.000000       5.000000       5.000000                      5.0   
mean    3.167835       0.55

In [ ]:
my_metrics = []
for seed in seeds:
    wandb.init(group='template_xshort', name=f'template_xshort_{seed}')

    template = (
        "<document>",
        "L'appello è <mask>"
    )

    best_train_args.seed = seed

    prompts_df = prepare_tokenization_dataframe(tokenizer, df, template)
    tokenized_prompts = tokenize_prompts(tokenizer, prompts_df)
    dataset = VJPDataset(tokenized_prompts)
    train_set, validation_set, test_set = torch.utils.data.random_split(
        dataset, [0.4, 0.3, 0.3], torch.Generator().manual_seed(17))

    trainer = Trainer(args=best_train_args, train_dataset=train_set,
                      eval_dataset=validation_set,
                      compute_metrics=metrics,
                      model_init=model_init)
    trainer.train()
    my_metrics.append(trainer.predict(validation_set).metrics)
    gc.collect()
    torch.cuda.empty_cache()

metrics_df = pd.DataFrame(my_metrics)
print(metrics_df)
print(metrics_df.describe())

eval/accuracy,▁█▇▄█▇▇▅▅▅▄▆▆▄▇▆
eval/f1_macro,▁▅▃▆▂▁▅▇▇▅▆▇█▅█▇
eval/loss,▁▁▁▁▃▂▂▂▅▅▆▇▇▇██
eval/runtime,▆▄▂▃▁▂▂▂▃▄█▇███▆
eval/samples_per_second,▃▅▇▆█▇▇▆▆▄▁▂▁▁▁▃
eval/steps_per_second,▃▅▇▆█▇▇▆▆▄▁▂▁▁▁▃
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▃▃▃▄▅▂▃▂▂▃▃▄▄▂▂▂▂▁▁▁▄▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,0.772400,0.916343,0.393939,0.318182,0.000000
2,0.566600,0.811300,0.575758,0.396078,0.000000
3,0.123900,1.172506,0.621212,0.383178,0.000000
4,1.337800,1.290113,0.621212,0.383178,0.000000
5,0.635600,0.855464,0.484848,0.477167,0.000000
6,0.649400,1.040423,0.454545,0.446412,0.000000
7,0.363500,1.187656,0.454545,0.452535,0.000000
8,0.009000,1.619544,0.530303,0.496926,0.000000
9,0.305500,2.124696,0.515152,0.498575,0.000000
10,0.000100,3.202300,0.590909,0.493030,0.000000


eval/accuracy,▁▇██▄▃▃▅▅▇▇█▆▇▇▇
eval/f1_macro,▁▃▃▃▅▄▄▆▆▆▇█▆▇▇▇
eval/loss,▁▁▂▂▁▂▂▃▄▇▇█████
eval/runtime,▅▁▂▁▂▁▂▂▄▆█▇█▇▇▇
eval/samples_per_second,▃█▇█▇█▇▇▅▃▁▂▁▂▂▂
eval/steps_per_second,▃█▇█▇█▇▇▅▃▁▂▁▂▂▂
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▃▄▃▂▂█▂▂▂▂▃▂▂▂▃▃▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,1.242300,1.231276,0.621212,0.383178,0.000000
2,0.435100,0.732396,0.530303,0.451622,0.000000
3,0.529300,0.882523,0.530303,0.487860,0.000000
4,0.349000,1.027199,0.500000,0.494312,0.000000
5,2.402400,1.828917,0.575758,0.365385,0.000000
6,0.393000,1.972033,0.590909,0.430853,0.000000
7,0.004800,2.032737,0.500000,0.430886,0.000000
8,0.069800,2.472914,0.439394,0.416766,0.000000
9,0.001900,3.308186,0.515152,0.441270,0.000000
10,2.143100,3.803734,0.515152,0.408735,0.000000


eval/accuracy,█▄▄▃▆▇▃▁▄▄▄▄▄▄▃▃
eval/f1_macro,▂▅▇▇▁▄▄▃▅▃▄█▄▄▅▅
eval/loss,▂▁▁▂▃▄▄▅▇█▇▇████
eval/runtime,▄▃▄▁▄▇▆▅██▇▇▆▄▁▁
eval/samples_per_second,▅▆▅█▅▂▃▃▁▁▂▂▃▅██
eval/steps_per_second,▅▆▅█▅▂▃▃▁▁▂▂▃▅██
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▅▅▄██▄▇▄▆▃▅▂▄▄█▁▁▂▁▁▁▃▂▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,0.441700,0.726223,0.439394,0.408860,0.000000
2,0.652400,0.974288,0.363636,0.284091,0.000000
3,0.692400,0.854688,0.439394,0.408860,0.000000
4,1.058400,0.974343,0.439394,0.436158,0.000000
5,0.138100,1.441147,0.590909,0.371429,0.000000
6,0.132700,2.352454,0.606061,0.377358,0.000000
7,0.027800,1.759941,0.469697,0.421777,0.000000
8,0.070300,2.266479,0.439394,0.428772,0.000000
9,0.020600,3.074278,0.454545,0.452535,0.000000
10,0.008000,3.400954,0.545455,0.488636,0.000000


   test_loss  test_accuracy  test_f1_macro  test_wrong_tokens_ratio  \
0   3.745296       0.621212       0.586984                      0.0   
1   3.228230       0.621212       0.586984                      0.0   
2   3.112008       0.575758       0.565789                      0.0   
3   3.640884       0.530303       0.511345                      0.0   
4   4.192459       0.575758       0.497826                      0.0   

   test_runtime  test_samples_per_second  test_steps_per_second  
0        7.2176                    9.144                  1.247  
1        7.1828                    9.189                  1.253  
2        7.2370                    9.120                  1.244  
3        7.0947                    9.303                  1.269  
4        7.0742                    9.330                  1.272  
       test_loss  test_accuracy  test_f1_macro  test_wrong_tokens_ratio  \
count   5.000000       5.000000       5.000000                      5.0   
mean    3.583775       0.58

In [ ]:
my_metrics = []
for seed in seeds:
    wandb.init(group='template_null', name=f'template_null_{seed}')

    template = (
        "<document>",
        "<mask>"
    )

    best_train_args.seed = seed

    prompts_df = prepare_tokenization_dataframe(tokenizer, df, template)
    tokenized_prompts = tokenize_prompts(tokenizer, prompts_df)
    dataset = VJPDataset(tokenized_prompts)
    train_set, validation_set, test_set = torch.utils.data.random_split(
        dataset, [0.4, 0.3, 0.3], torch.Generator().manual_seed(17))

    trainer = Trainer(args=best_train_args, train_dataset=train_set,
                      eval_dataset=validation_set,
                      compute_metrics=metrics,
                      model_init=model_init)
    trainer.train()
    my_metrics.append(trainer.predict(validation_set).metrics)
    gc.collect()
    torch.cuda.empty_cache()

metrics_df = pd.DataFrame(my_metrics)
print(metrics_df)
print(metrics_df.describe())

wandb: Currently logged in as: ballman. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,0.738900,0.815219,0.621212,0.383178,0.000000
2,3.170800,1.656092,0.378788,0.274725,0.000000
3,0.124400,0.952898,0.621212,0.383178,0.000000
4,3.029800,1.842044,0.621212,0.383178,0.000000
5,0.610000,0.717004,0.545455,0.545037,0.000000
6,0.288900,0.830805,0.469697,0.466636,0.000000
7,1.305400,1.408167,0.378788,0.274725,0.000000
8,0.779300,1.477357,0.378788,0.274725,0.000000
9,0.324000,1.267557,0.409091,0.355695,0.000000
10,0.983100,1.869046,0.409091,0.342529,0.000000


eval/accuracy,█▁██▆▄▁▁▂▂▆▇▇▆▄▅
eval/f1_macro,▄▁▄▄█▆▁▁▃▃█▆▇▅▆▇
eval/loss,▂▇▂█▁▂▅▆▄█▃▄▅▇██
eval/runtime,▂▁▆▅▆▇█▇▆▆▆▆█▇▇▇
eval/samples_per_second,▇█▃▄▂▂▁▂▃▃▃▃▁▂▂▂
eval/steps_per_second,▇█▃▄▂▂▁▂▃▃▃▃▁▂▂▂
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▃▄▂▂▃▂▂▂▃▆▂▂▂▂▂▆▂▁▂▂▁▂▁▂▄▁▂▂▂▂▁▁▂▁▁▂▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,2.115200,2.183225,0.621212,0.383178,0.000000
2,1.360100,0.685035,0.606061,0.410714,0.000000
3,0.801500,0.705113,0.454545,0.428846,0.000000
4,1.220500,0.963830,0.409091,0.367101,0.000000
5,1.688000,1.061779,0.621212,0.383178,0.000000
6,0.746800,0.867530,0.606061,0.377358,0.000000
7,2.857700,1.224176,0.393939,0.318182,0.000000
8,0.384300,1.265633,0.621212,0.383178,0.000000
9,0.391200,1.494455,0.606061,0.377358,0.000000
10,0.995800,1.223222,0.545455,0.476190,0.000000


eval/accuracy,██▃▁██▁██▆▃▆▇▅▅▅
eval/f1_macro,▃▄▅▃▃▃▁▃▃▆▆█▇▆▇▇
eval/loss,▆▁▁▂▂▂▃▃▄▃▅▄▆▇██
eval/runtime,█▆█▃▆▄▃▃▂▁▂▂▁▂▁▁
eval/samples_per_second,▁▃▁▆▂▄▆▆▇█▇▇█▇██
eval/steps_per_second,▁▃▁▆▃▄▆▆▇█▇▇█▇██
eval/wrong_tokens_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▄▆▃█▃▄▃▂▃▃▂▃▃▅▂▃▅▃▄▁▂▄▁▃▂▁▁▇▂▂▁▁▂▁▁▁▁▁▁▁


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Wrong Tokens Ratio
1,1.137200,0.847683,0.621212,0.383178,0.000000
2,1.578300,0.704336,0.636364,0.524610,0.000000
3,0.551800,0.759781,0.621212,0.383178,0.000000
4,1.141200,0.722315,0.545455,0.517073,0.000000
5,0.553800,0.754058,0.606061,0.503472,0.000000
6,0.387700,0.769324,0.606061,0.484994,0.000000
7,0.686300,1.439239,0.606061,0.410714,0.000000
8,0.009000,1.354371,0.469697,0.466636,0.000000
9,0.022000,1.502277,0.606061,0.587500,0.000000
10,0.016300,1.655462,0.606061,0.574405,0.000000


   test_loss  test_accuracy  test_f1_macro  test_wrong_tokens_ratio  \
0   1.075598       0.560606       0.548052                      0.0   
1   1.319856       0.500000       0.490526                      0.0   
2   1.662842       0.621212       0.557759                      0.0   
3   1.551896       0.545455       0.529915                      0.0   
4   1.502277       0.606061       0.587500                      0.0   

   test_runtime  test_samples_per_second  test_steps_per_second  
0        7.2135                    9.149                  1.248  
1        7.6744                    8.600                  1.173  
2        7.2869                    9.057                  1.235  
3        7.6268                    8.654                  1.180  
4        7.6868                    8.586                  1.171  
       test_loss  test_accuracy  test_f1_macro  test_wrong_tokens_ratio  \
count   5.000000       5.000000       5.000000                      5.0   
mean    1.422494       0.56

# Bibliography

[Gao et al., 2020] Tianyu Gao, Adam Fisch, and Danqi Chen. Making pre-trained language models better few-shot
learners. arXiv preprint arXiv:2012.15723, 2020.

[Galli et al., 2022] Federico Galli, Giulia Grundler, Alessia Fidelangeli, An-
drea Galassi, Francesca Lagioia, Elena Palmieri, Federico Ruggeri, Giovanni Sartor, and Paolo Torroni. 2022. Predicting outcomes of italian vat decisions 1. In Legal Knowledge and Information Systems, pages 188–193. IOS Press.

[Logan IV et al., 2022] Robert Logan IV, Ivana Balazevic, Eric Wallace, Fabio Petroni, Sameer Singh, and Sebastian Riedel. 2022. Cutting Down on Prompts and Parameters: Simple Few-Shot Learning with Language Models. In Findings of the Association for Computational Linguistics: ACL 2022, pages 2824–2835, Dublin, Ireland. Association for Computational Linguistics.